In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import LSTM
from tensorflow.keras.metrics import *
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 0s 61ms/step - loss: 0.7471 - accuracy: 0.5170 - precision: 0.4828 - recall: 0.4252 - auc: 0.5038
Epoch 2/50
8/8 [==============================] - 1s 77ms/step - loss: 0.7112 - accuracy: 0.5203 - precision: 0.4893 - recall: 0.5327 - auc: 0.5209
Epoch 3/50
8/8 [==============================] - 1s 71ms/step - loss: 0.7201 - accuracy: 0.4863 - precision: 0.4491 - recall: 0.4229 - auc: 0.4907
Epoch 4/50
8/8 [==============================] - 0s 62ms/step - loss: 0.7202 - accuracy: 0.4863 - precision: 0.4494 - recall: 0.4252 - auc: 0.4799
Epoch 5/50
8/8 [==============================] - 1s 74ms/step - loss: 0.7098 - accuracy: 0.5049 - precision: 0.4701 - recall: 0.4416 - auc: 0.5019
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 0.7098 - accuracy: 0.5016 - precision: 0.4554 - recall: 0.3224 - auc: 0.4892
Epoch 7/50
8/8 [==============================] - 0s 61ms/step - loss: 0.7100 - accuracy: 0.5027 - precision: 0.

8/8 [==============================] - 1s 75ms/step - loss: 0.7306 - accuracy: 0.4765 - precision_1: 0.4399 - recall_1: 0.4276 - auc_1: 0.4658
Epoch 6/50
8/8 [==============================] - 1s 72ms/step - loss: 0.7177 - accuracy: 0.4951 - precision_1: 0.4433 - recall_1: 0.3014 - auc_1: 0.4796
Epoch 7/50
8/8 [==============================] - 1s 73ms/step - loss: 0.7089 - accuracy: 0.5301 - precision_1: 0.4977 - recall_1: 0.2547 - auc_1: 0.5132
Epoch 8/50
8/8 [==============================] - 1s 63ms/step - loss: 0.7156 - accuracy: 0.5137 - precision_1: 0.4570 - recall_1: 0.1986 - auc_1: 0.4791
Epoch 9/50
8/8 [==============================] - 1s 68ms/step - loss: 0.7132 - accuracy: 0.4754 - precision_1: 0.4380 - recall_1: 0.4206 - auc_1: 0.4723
Epoch 10/50
8/8 [==============================] - 1s 80ms/step - loss: 0.7050 - accuracy: 0.5060 - precision_1: 0.4779 - recall_1: 0.5818 - auc_1: 0.5199
Epoch 11/50
8/8 [==============================] - 1s 63ms/step - loss: 0.7078 - accur

8/8 [==============================] - 0s 53ms/step - loss: 0.6981 - accuracy: 0.5323 - precision_2: 0.5015 - recall_2: 0.3832 - auc_2: 0.5173
Epoch 9/50
8/8 [==============================] - 0s 60ms/step - loss: 0.7021 - accuracy: 0.5093 - precision_2: 0.4718 - recall_2: 0.3902 - auc_2: 0.5045
Epoch 10/50
8/8 [==============================] - 0s 54ms/step - loss: 0.7007 - accuracy: 0.5060 - precision_2: 0.4585 - recall_2: 0.2967 - auc_2: 0.5045
Epoch 11/50
8/8 [==============================] - 0s 60ms/step - loss: 0.7033 - accuracy: 0.5203 - precision_2: 0.4819 - recall_2: 0.3107 - auc_2: 0.5079
Epoch 12/50
8/8 [==============================] - 0s 58ms/step - loss: 0.6957 - accuracy: 0.5301 - precision_2: 0.4983 - recall_2: 0.3388 - auc_2: 0.5157
Epoch 13/50
8/8 [==============================] - 0s 62ms/step - loss: 0.6996 - accuracy: 0.5005 - precision_2: 0.4514 - recall_2: 0.3037 - auc_2: 0.4997
Epoch 14/50
8/8 [==============================] - 1s 73ms/step - loss: 0.6935 - ac

8/8 [==============================] - 0s 57ms/step - loss: 0.6915 - accuracy: 0.5531 - precision_3: 0.5294 - recall_3: 0.4206 - auc_3: 0.5566
Epoch 11/50
8/8 [==============================] - 0s 57ms/step - loss: 0.6819 - accuracy: 0.5750 - precision_3: 0.5498 - recall_3: 0.5164 - auc_3: 0.5869
Epoch 12/50
8/8 [==============================] - 0s 62ms/step - loss: 0.6944 - accuracy: 0.5356 - precision_3: 0.5061 - recall_3: 0.3855 - auc_3: 0.5538
Epoch 13/50
8/8 [==============================] - 1s 63ms/step - loss: 0.6724 - accuracy: 0.5849 - precision_3: 0.5773 - recall_3: 0.4276 - auc_3: 0.6137
Epoch 14/50
8/8 [==============================] - 0s 60ms/step - loss: 0.6508 - accuracy: 0.6166 - precision_3: 0.6175 - recall_3: 0.4790 - auc_3: 0.6588
Epoch 15/50
8/8 [==============================] - 0s 60ms/step - loss: 0.6271 - accuracy: 0.6670 - precision_3: 0.6520 - recall_3: 0.6215 - auc_3: 0.7207
Epoch 16/50
8/8 [==============================] - 0s 58ms/step - loss: 0.5920 - a

Epoch 13/50
8/8 [==============================] - 0s 58ms/step - loss: 0.7016 - accuracy: 0.5170 - precision_4: 0.4733 - recall_4: 0.2687 - auc_4: 0.4900
Epoch 14/50
8/8 [==============================] - 0s 55ms/step - loss: 0.7029 - accuracy: 0.5027 - precision_4: 0.4519 - recall_4: 0.2850 - auc_4: 0.4939
Epoch 15/50
8/8 [==============================] - 0s 58ms/step - loss: 0.7004 - accuracy: 0.5137 - precision_4: 0.4652 - recall_4: 0.2500 - auc_4: 0.5048
Epoch 16/50
8/8 [==============================] - 1s 69ms/step - loss: 0.6993 - accuracy: 0.5203 - precision_4: 0.4826 - recall_4: 0.3248 - auc_4: 0.5046
Epoch 17/50
8/8 [==============================] - 1s 63ms/step - loss: 0.6999 - accuracy: 0.5181 - precision_4: 0.4771 - recall_4: 0.2921 - auc_4: 0.4984
Epoch 18/50
8/8 [==============================] - 1s 64ms/step - loss: 0.6916 - accuracy: 0.5334 - precision_4: 0.5045 - recall_4: 0.2617 - auc_4: 0.5328
Epoch 19/50
8/8 [==============================] - 1s 75ms/step - loss

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3),input_shape=(None,n_length,n_features)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)
    
run_experiment_2(X,y)

Epoch 1/50
8/8 [==============================] - 1s 82ms/step - loss: 0.7456 - accuracy: 0.5115 - precision_5: 0.4763 - recall_5: 0.4229 - auc_5: 0.5121
Epoch 2/50
8/8 [==============================] - 1s 71ms/step - loss: 0.7217 - accuracy: 0.5422 - precision_5: 0.5130 - recall_5: 0.4626 - auc_5: 0.5436
Epoch 3/50
8/8 [==============================] - 1s 67ms/step - loss: 0.7283 - accuracy: 0.5005 - precision_5: 0.4659 - recall_5: 0.4463 - auc_5: 0.4953
Epoch 4/50
8/8 [==============================] - 1s 73ms/step - loss: 0.7369 - accuracy: 0.4589 - precision_5: 0.4149 - recall_5: 0.3762 - auc_5: 0.4570
Epoch 5/50
8/8 [==============================] - 0s 62ms/step - loss: 0.7193 - accuracy: 0.5071 - precision_5: 0.4707 - recall_5: 0.4136 - auc_5: 0.5001
Epoch 6/50
8/8 [==============================] - 1s 67ms/step - loss: 0.6921 - accuracy: 0.5761 - precision_5: 0.5627 - recall_5: 0.4299 - auc_5: 0.5753
Epoch 7/50
8/8 [==============================] - 1s 77ms/step - loss: 0.689

Epoch 1/50
8/8 [==============================] - 1s 74ms/step - loss: 0.7340 - accuracy: 0.5060 - precision_6: 0.4637 - recall_6: 0.3435 - auc_6: 0.4951
Epoch 2/50
8/8 [==============================] - 1s 97ms/step - loss: 0.7187 - accuracy: 0.4962 - precision_6: 0.4524 - recall_6: 0.3551 - auc_6: 0.4929
Epoch 3/50
8/8 [==============================] - 1s 80ms/step - loss: 0.7270 - accuracy: 0.5060 - precision_6: 0.4743 - recall_6: 0.4953 - auc_6: 0.4798
Epoch 4/50
8/8 [==============================] - 1s 79ms/step - loss: 0.7116 - accuracy: 0.5126 - precision_6: 0.4845 - recall_6: 0.6215 - auc_6: 0.5233
Epoch 5/50
8/8 [==============================] - 1s 78ms/step - loss: 0.6857 - accuracy: 0.5411 - precision_6: 0.5122 - recall_6: 0.4416 - auc_6: 0.5737
Epoch 6/50
8/8 [==============================] - 1s 70ms/step - loss: 0.6840 - accuracy: 0.5531 - precision_6: 0.5296 - recall_6: 0.4182 - auc_6: 0.5769
Epoch 7/50
8/8 [==============================] - 1s 72ms/step - loss: 0.677

Epoch 1/50
8/8 [==============================] - 0s 55ms/step - loss: 0.7132 - accuracy: 0.5334 - precision_7: 0.5021 - recall_7: 0.5537 - auc_7: 0.5411
Epoch 2/50
8/8 [==============================] - 0s 52ms/step - loss: 0.7065 - accuracy: 0.5235 - precision_7: 0.4894 - recall_7: 0.3762 - auc_7: 0.5353
Epoch 3/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7025 - accuracy: 0.5192 - precision_7: 0.4860 - recall_7: 0.4463 - auc_7: 0.5430
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6937 - accuracy: 0.5553 - precision_7: 0.5327 - recall_7: 0.4182 - auc_7: 0.5633
Epoch 5/50
8/8 [==============================] - 0s 52ms/step - loss: 0.6762 - accuracy: 0.5739 - precision_7: 0.5484 - recall_7: 0.5164 - auc_7: 0.6051
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 0.6657 - accuracy: 0.5641 - precision_7: 0.5517 - recall_7: 0.3738 - auc_7: 0.6233
Epoch 7/50
8/8 [==============================] - 0s 55ms/step - loss: 0.651

Epoch 1/50
8/8 [==============================] - 0s 53ms/step - loss: 0.7340 - accuracy: 0.5192 - precision_8: 0.4858 - recall_8: 0.4393 - auc_8: 0.5105
Epoch 2/50
8/8 [==============================] - 0s 52ms/step - loss: 0.7119 - accuracy: 0.5301 - precision_8: 0.4983 - recall_8: 0.3458 - auc_8: 0.5101
Epoch 3/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7095 - accuracy: 0.4940 - precision_8: 0.4562 - recall_8: 0.4136 - auc_8: 0.5012
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7027 - accuracy: 0.5345 - precision_8: 0.5059 - recall_8: 0.3014 - auc_8: 0.5351
Epoch 5/50
8/8 [==============================] - 0s 59ms/step - loss: 0.6876 - accuracy: 0.5586 - precision_8: 0.5371 - recall_8: 0.4229 - auc_8: 0.5652
Epoch 6/50
8/8 [==============================] - 0s 62ms/step - loss: 0.6893 - accuracy: 0.5312 - precision_8: 0.5000 - recall_8: 0.4720 - auc_8: 0.5606
Epoch 7/50
8/8 [==============================] - 0s 60ms/step - loss: 0.683

Epoch 1/50
8/8 [==============================] - 0s 55ms/step - loss: 0.7466 - accuracy: 0.5060 - precision_9: 0.4738 - recall_9: 0.4860 - auc_9: 0.4974
Epoch 2/50
8/8 [==============================] - 0s 51ms/step - loss: 0.7341 - accuracy: 0.5246 - precision_9: 0.4894 - recall_9: 0.3248 - auc_9: 0.5006
Epoch 3/50
8/8 [==============================] - 0s 55ms/step - loss: 0.7193 - accuracy: 0.5192 - precision_9: 0.4804 - recall_9: 0.3154 - auc_9: 0.5171
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7037 - accuracy: 0.5356 - precision_9: 0.5040 - recall_9: 0.5818 - auc_9: 0.5467
Epoch 5/50
8/8 [==============================] - 0s 52ms/step - loss: 0.7073 - accuracy: 0.5159 - precision_9: 0.4839 - recall_9: 0.4907 - auc_9: 0.5203: 0s - loss: 0.7078 - accuracy: 0.5130 - precision_9: 0.4717 - recall_9: 0.4958 - auc_9: 0.52
Epoch 6/50
8/8 [==============================] - 0s 55ms/step - loss: 0.6976 - accuracy: 0.5235 - precision_9: 0.4912 - recall_9: 0.4579

 Accuracy: 68.734% (+/-7.039) 
 Precision: 61.607% (+/-11.186) 
 Recall: 52.273% (+/-25.430) 
 AUC: 78.134% (+/-7.424) 


In [13]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)
    
run_experiment_3(X,y)

Epoch 1/50
8/8 [==============================] - 0s 52ms/step - loss: 0.7762 - accuracy: 0.5279 - precision_10: 0.4963 - recall_10: 0.4696 - auc_10: 0.5148
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7192 - accuracy: 0.5498 - precision_10: 0.5191 - recall_10: 0.5397 - auc_10: 0.5525
Epoch 3/50
8/8 [==============================] - 0s 51ms/step - loss: 0.6814 - accuracy: 0.5871 - precision_10: 0.5726 - recall_10: 0.4696 - auc_10: 0.6133
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6611 - accuracy: 0.5849 - precision_10: 0.5525 - recall_10: 0.6028 - auc_10: 0.6278
Epoch 5/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6618 - accuracy: 0.5772 - precision_10: 0.5449 - recall_10: 0.5958 - auc_10: 0.6239
Epoch 6/50
8/8 [==============================] - 0s 53ms/step - loss: 0.6223 - accuracy: 0.6342 - precision_10: 0.6211 - recall_10: 0.5631 - auc_10: 0.6995
Epoch 7/50
8/8 [==============================] - 0s 50ms/

Epoch 1/50
8/8 [==============================] - 1s 125ms/step - loss: 0.8029 - accuracy: 0.4995 - precision_11: 0.4645 - recall_11: 0.4439 - auc_11: 0.4949
Epoch 2/50
8/8 [==============================] - 1s 131ms/step - loss: 0.7193 - accuracy: 0.5455 - precision_11: 0.5164 - recall_11: 0.4790 - auc_11: 0.5608
Epoch 3/50
8/8 [==============================] - 1s 96ms/step - loss: 0.7321 - accuracy: 0.5214 - precision_11: 0.4889 - recall_11: 0.4650 - auc_11: 0.5371
Epoch 4/50
8/8 [==============================] - 1s 104ms/step - loss: 0.7005 - accuracy: 0.5630 - precision_11: 0.5420 - recall_11: 0.4369 - auc_11: 0.5894
Epoch 5/50
8/8 [==============================] - 1s 100ms/step - loss: 0.6805 - accuracy: 0.5827 - precision_11: 0.5491 - recall_11: 0.6145 - auc_11: 0.6193
Epoch 6/50
8/8 [==============================] - 1s 111ms/step - loss: 0.6564 - accuracy: 0.5969 - precision_11: 0.5888 - recall_11: 0.4650 - auc_11: 0.6526
Epoch 7/50
8/8 [==============================] - 1s 

8/8 [==============================] - 0s 49ms/step - loss: 0.0181 - accuracy: 0.9978 - precision_11: 0.9977 - recall_11: 0.9977 - auc_11: 0.9998
Epoch 1/50
8/8 [==============================] - 1s 81ms/step - loss: 0.8414 - accuracy: 0.4885 - precision_12: 0.4651 - recall_12: 0.6075 - auc_12: 0.5001
Epoch 2/50
8/8 [==============================] - 1s 99ms/step - loss: 0.7449 - accuracy: 0.5411 - precision_12: 0.5213 - recall_12: 0.2570 - auc_12: 0.5393
Epoch 3/50
8/8 [==============================] - 1s 94ms/step - loss: 0.7342 - accuracy: 0.5301 - precision_12: 0.4990 - recall_12: 0.5607 - auc_12: 0.5354
Epoch 4/50
8/8 [==============================] - 1s 64ms/step - loss: 0.7203 - accuracy: 0.5531 - precision_12: 0.5214 - recall_12: 0.5701 - auc_12: 0.5473
Epoch 5/50
8/8 [==============================] - 1s 82ms/step - loss: 0.7183 - accuracy: 0.5301 - precision_12: 0.4981 - recall_12: 0.3061 - auc_12: 0.5502
Epoch 6/50
8/8 [==============================] - 1s 76ms/step - loss

8/8 [==============================] - 1s 76ms/step - loss: 0.0810 - accuracy: 0.9704 - precision_12: 0.9652 - recall_12: 0.9720 - auc_12: 0.9958
Epoch 47/50
8/8 [==============================] - 1s 74ms/step - loss: 0.0787 - accuracy: 0.9726 - precision_12: 0.9741 - recall_12: 0.9673 - auc_12: 0.9958
Epoch 48/50
8/8 [==============================] - 1s 76ms/step - loss: 0.0986 - accuracy: 0.9715 - precision_12: 0.9548 - recall_12: 0.9860 - auc_12: 0.9940
Epoch 49/50
8/8 [==============================] - 0s 53ms/step - loss: 0.1241 - accuracy: 0.9606 - precision_12: 0.9876 - recall_12: 0.9276 - auc_12: 0.9907
Epoch 50/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1213 - accuracy: 0.9485 - precision_12: 0.9062 - recall_12: 0.9930 - auc_12: 0.9937
Epoch 1/50
8/8 [==============================] - 1s 74ms/step - loss: 0.7760 - accuracy: 0.5356 - precision_13: 0.5051 - recall_13: 0.4673 - auc_13: 0.5368
Epoch 2/50
8/8 [==============================] - 1s 90ms/step - 

8/8 [==============================] - 1s 77ms/step - loss: 0.0618 - accuracy: 0.9770 - precision_13: 0.9722 - recall_13: 0.9790 - auc_13: 0.9968
Epoch 44/50
8/8 [==============================] - 1s 79ms/step - loss: 0.0527 - accuracy: 0.9847 - precision_13: 0.9792 - recall_13: 0.9883 - auc_13: 0.9974
Epoch 45/50
8/8 [==============================] - 1s 68ms/step - loss: 0.0589 - accuracy: 0.9825 - precision_13: 0.9928 - recall_13: 0.9696 - auc_13: 0.9970
Epoch 46/50
8/8 [==============================] - 1s 75ms/step - loss: 0.0538 - accuracy: 0.9847 - precision_13: 0.9836 - recall_13: 0.9836 - auc_13: 0.9972
Epoch 47/50
8/8 [==============================] - 1s 77ms/step - loss: 0.0560 - accuracy: 0.9814 - precision_13: 0.9790 - recall_13: 0.9813 - auc_13: 0.9972
Epoch 48/50
8/8 [==============================] - 1s 75ms/step - loss: 0.0357 - accuracy: 0.9912 - precision_13: 0.9976 - recall_13: 0.9836 - auc_13: 0.9992
Epoch 49/50
8/8 [==============================] - 0s 50ms/step 

8/8 [==============================] - 1s 73ms/step - loss: 0.0988 - accuracy: 0.9660 - precision_14: 0.9760 - recall_14: 0.9509 - auc_14: 0.9935
Epoch 41/50
8/8 [==============================] - 1s 84ms/step - loss: 0.0890 - accuracy: 0.9693 - precision_14: 0.9651 - recall_14: 0.9696 - auc_14: 0.9951
Epoch 42/50
8/8 [==============================] - 1s 77ms/step - loss: 0.0671 - accuracy: 0.9770 - precision_14: 0.9700 - recall_14: 0.9813 - auc_14: 0.9971
Epoch 43/50
8/8 [==============================] - 1s 86ms/step - loss: 0.0624 - accuracy: 0.9814 - precision_14: 0.9813 - recall_14: 0.9790 - auc_14: 0.9968
Epoch 44/50
8/8 [==============================] - 1s 73ms/step - loss: 0.0737 - accuracy: 0.9770 - precision_14: 0.9615 - recall_14: 0.9907 - auc_14: 0.9961
Epoch 45/50
8/8 [==============================] - 1s 96ms/step - loss: 0.0704 - accuracy: 0.9759 - precision_14: 0.9677 - recall_14: 0.9813 - auc_14: 0.9955
Epoch 46/50
8/8 [==============================] - 1s 67ms/step 